In [2]:
!pip install vk_api
!pip install ipywidgets
!pip install jupyter

In [3]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [4]:
!pip install pandas

In [5]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

python3 -m venv env

source env/bin/activate

In [6]:
import vk_api
import requests
import shutil
import os
import json
from ipywidgets import widgets  

In [7]:
output_text = widgets.Output
@interact
def getVkApi(login="89993492809", password="Mirazh86", token="a749c3b8839839ce29d4278fc6a893493ff949c920565107718c123f52084f3e2f1d6a5e7415816085a18"):
    vk_session = vk_api.VkApi(login=login, 
                          password=password, 
                          token=token)
    output_text.outputs=vk_session.get_api()

interactive(children=(Text(value='89993492809', description='login'), Text(value='Mirazh86', description='pass…

In [8]:
vk = output_text.outputs

In [9]:
publics = vk.groups.search(q='Подслушано', filter='group', count=50)
public_ids = []
for public in publics['items']:
  public_ids.append(public['id'])

In [10]:
def download_image(image_url, path, no):
#   print(image_url, image_url.split("."), image_url.split(".")[-2])  
    destname = path + '/' + no + '.'  + image_url.split(".")[-1].split("?")[0]
    # filename = path + '/' + image_url.split("/")[-1]
    r = requests.get(image_url, stream = True)
    if r.status_code == 200:
        r.raw.decode_content = True
        # fdst = open(destname, 'w')
        with open(destname,'wb') as f:
            shutil.copyfileobj(r.raw, f)
    else:
        print('Image Couldn\'t be retreived')

In [11]:
public_info = {}

for index, id in enumerate(public_ids):
  path = f'./ParsedImages/{id}'
  isExist = os.path.exists(path)
  if not isExist:
    os.makedirs(path)
    print("Created dir:", path)
  wall = vk.wall.get(owner_id=-id, count=1000)['items']
  print(f'Группа №{index+1}/50')
  comment_number = []
  for index1, post in enumerate(wall):
    try:
      comment_number.append(post['comments']['count'])
      if post['date'] > 1640984400:
        download_image(post['attachments'][0]['photo']['sizes'][-1]['url'], 
                       path, str(index1))
    except Exception as E:
      # print(E)
      pass
  # TODO: метрика любви к котам
  #  подогнать под трешхолд то, что стоит отнести к не котам/животным
  try:
    public_info[id] = {'mean_comments_number': sum(comment_number)/(len(comment_number)+1),
                       'subscribers_number': vk.groups.getMembers(group_id=id)['count']}
  except Exception:
    public_info[id] = {'mean_comments_number': None,
                      'subscribers_number': None}

Created dir: ./ParsedImages/121503864
Группа №1/50
Created dir: ./ParsedImages/58741668
Группа №2/50
Created dir: ./ParsedImages/108718137
Группа №3/50
Created dir: ./ParsedImages/196994092
Группа №4/50
Created dir: ./ParsedImages/161945823
Группа №5/50
Created dir: ./ParsedImages/114347775
Группа №6/50
Created dir: ./ParsedImages/63771931
Группа №7/50
Created dir: ./ParsedImages/120161460
Группа №8/50
Created dir: ./ParsedImages/152826935
Группа №9/50
Created dir: ./ParsedImages/62210730
Группа №10/50
Created dir: ./ParsedImages/180044739
Группа №11/50
Created dir: ./ParsedImages/67238316
Группа №12/50
Created dir: ./ParsedImages/131886625
Группа №13/50
Created dir: ./ParsedImages/53115317
Группа №14/50
Created dir: ./ParsedImages/79010330
Группа №15/50
Created dir: ./ParsedImages/59087769
Группа №16/50
Created dir: ./ParsedImages/133862163
Группа №17/50
Created dir: ./ParsedImages/61403518
Группа №18/50
Created dir: ./ParsedImages/38981260
Группа №19/50
Created dir: ./ParsedImages/69

In [12]:
with open("./public_info.json", "w") as f:
    json.dump(public_info , f) 